##Install Caffe2

In [1]:
!wget https://anaconda.org/pytorch/pytorch-nightly/1.0.0.dev20181206/download/linux-64/pytorch-nightly-1.0.0.dev20181206-py2.7_cuda9.2.148_cudnn7.4.1_0.tar.bz2
!tar xvjf pytorch-nightly-1.0.0.dev20181206-py2.7_cuda9.2.148_cudnn7.4.1_0.tar.bz2
!cp -r lib/python2.7/site-packages/* /usr/local/lib/python2.7/dist-packages/

--2018-12-07 12:54:14--  https://anaconda.org/pytorch/pytorch-nightly/1.0.0.dev20181206/download/linux-64/pytorch-nightly-1.0.0.dev20181206-py2.7_cuda9.2.148_cudnn7.4.1_0.tar.bz2
Resolving anaconda.org (anaconda.org)... 104.16.3.213, 104.16.5.213, 104.16.2.213, ...
Connecting to anaconda.org (anaconda.org)|104.16.3.213|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://binstar-cio-packages-prod.s3.amazonaws.com/5b9c2539ab12f91347072300/5c095b5837f3391343b78dff?response-content-disposition=attachment%3B%20filename%3D%22pytorch-nightly-1.0.0.dev20181206-py2.7_cuda9.2.148_cudnn7.4.1_0.tar.bz2%22%3B%20filename%2A%3DUTF-8%27%27pytorch-nightly-1.0.0.dev20181206-py2.7_cuda9.2.148_cudnn7.4.1_0.tar.bz2&response-content-type=application%2Fx-tar&X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Expires=60&X-Amz-Date=20181207T125415Z&X-Amz-SignedHeaders=host&X-Amz-Security-Token=FQoGZXIvYXdzEBoaDK9gpPLSfJJwwk%2F7gSK3A5ve5PvnmSVUh60kBo3if0Vs6zhJak3UyrLewhdJQe0hIRGu%2Fx0LjID

####Check Caffe2 installation

In [2]:
# To check if Caffe2 build was successful
!python -c 'from caffe2.python import core' 2>/dev/null && echo "Success" || echo "Failure"

# To check if Caffe2 GPU build was successful
# This must print a number > 0 in order to use Detectron
!python -c 'from caffe2.python import workspace; print(workspace.NumCudaDevices())'

Success
1


##Install Cocoapi

In [3]:
!apt-get install python-dev
!pip install cython
!pip install pycocotools
!git clone https://github.com/cocodataset/cocoapi.git
!cd cocoapi/PythonAPI && make install

import os
os.environ['COCOAPI'] = ":/content/cocoapi"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-dev is already the newest version (2.7.15~rc1-1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
    100% |████████████████████████████████| 2.0MB 10.1MB/s 
    100% |████████████████████████████████| 1.5MB 10.6MB/s 
  Running setup.py bdist_wheel for pycocotools ... - \ | / done
  Stored in directory: /root/.cache/pip/wheels/dc/e6/36/0e1ae88c868eb42d3f92181b1c9bbd0b217a7ec3da6bd62e55
Successfully built pycocotools
Cloning into 'cocoapi'...
remote: Enumerating objects: 947, done.
remote: Total 947 (delta 0), reused 0 (delta 0), pack-reused 947
Receiving objects: 100% (947/947), 11.69 MiB | 6.40 MiB/s, done.
Resolving deltas: 100% (565/565), done.
# install pycocotools to the Python site-packages
python setup.py build_ext install
running build_ext
cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
/usr/local/lib/python2.7/dist-packages/Cython/Compiler/Main.p

##Install Detectron

In [4]:
!git clone https://github.com/facebookresearch/detectron
!pip install -r detectron/requirements.txt
!cd detectron && make
!python detectron/detectron/tests/test_spatial_narrow_as_op.py
os.chdir('detectron')

Cloning into 'detectron'...
remote: Enumerating objects: 1172, done.
remote: Total 1172 (delta 0), reused 0 (delta 0), pack-reused 1172
Receiving objects: 100% (1172/1172), 4.19 MiB | 3.34 MiB/s, done.
Resolving deltas: 100% (731/731), done.
python setup.py develop --user
Compiling detectron/utils/cython_bbox.pyx because it changed.
Compiling detectron/utils/cython_nms.pyx because it changed.
[1/2] Cythonizing detectron/utils/cython_bbox.pyx
/usr/local/lib/python2.7/dist-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/detectron/detectron/utils/cython_bbox.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
[2/2] Cythonizing detectron/utils/cython_nms.pyx
/usr/local/lib/python2.7/dist-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/detectron

##Run Detectron on images in detectron/demo

In [5]:
!python tools/infer_simple.py \
   --cfg configs/12_2017_baselines/e2e_keypoint_rcnn_R-101-FPN_s1x.yaml \
   --output-dir demo/output \
   --image-ext jpg \
   --wts https://s3-us-west-2.amazonaws.com/detectron/37698009/12_2017_baselines/e2e_keypoint_rcnn_R-101-FPN_s1x.yaml.08_45_57.YkrJgP6O/output/train/keypoints_coco_2014_train:keypoints_coco_2014_valminusminival/generalized_rcnn/model_final.pkl \
   --output-ext jpg \
  demo

Found Detectron ops lib: /usr/local/lib/python2.7/dist-packages/torch/lib/libcaffe2_detectron_ops_gpu.so
[E init_intrinsics_check.cc:43] CPU feature avx is present on your machine, but the Caffe2 binary is not compiled with it. It means you may not get the full speed of your CPU.
[E init_intrinsics_check.cc:43] CPU feature avx2 is present on your machine, but the Caffe2 binary is not compiled with it. It means you may not get the full speed of your CPU.
[E init_intrinsics_check.cc:43] CPU feature fma is present on your machine, but the Caffe2 binary is not compiled with it. It means you may not get the full speed of your CPU.
INFO io.py:  91: Downloading remote file https://s3-us-west-2.amazonaws.com/detectron/37698009/12_2017_baselines/e2e_keypoint_rcnn_R-101-FPN_s1x.yaml.08_45_57.YkrJgP6O/output/train/keypoints_coco_2014_train:keypoints_coco_2014_valminusminival/generalized_rcnn/model_final.pkl to /tmp/detectron-download-cache/37698009/12_2017_baselines/e2e_keypoint_rcnn_R-101-FPN_s1

##Download files

In [0]:
from google.colab import files

for file in os.listdir('demo/output'):
  if file.endswith(('.jpg', '.pdf')):
    files.download('demo/output/'+file)